In [1]:
import os
import pandas as pd

In [2]:
projects = os.listdir('../projects/')

In [3]:
for project in projects:
    file_name = '{}.csv'.format(project)
    df_mutation = pd.read_csv('mutation/{}'.format(file_name))
    df_code_smells = pd.read_csv('code-smells/{}'.format(file_name))
    df_test_smells = pd.read_csv('test-smells/{}'.format(file_name))
    df_ck_metrics = pd.read_csv('ck-metrics/{}'.format(file_name))
    df_readability = pd.read_csv('readability/{}'.format(file_name))
    print('~~~~~~~~~~~~~~~~~~ {} ~~~~~~~~~~~~~~~~'.format(project))
    print(df_mutation)
    print(df_ck_metrics)
    print(df_code_smells)
    print(df_test_smells)
    print(df_readability)

~~~~~~~~~~~~~~~~~~ commons-numbers ~~~~~~~~~~~~~~~~
            project                         module  \
0   commons-numbers     commons-numbers-quaternion   
1   commons-numbers     commons-numbers-quaternion   
2   commons-numbers           commons-numbers-core   
3   commons-numbers           commons-numbers-core   
4   commons-numbers        commons-numbers-complex   
5   commons-numbers         commons-numbers-primes   
6   commons-numbers         commons-numbers-primes   
7   commons-numbers          commons-numbers-gamma   
8   commons-numbers          commons-numbers-gamma   
9   commons-numbers          commons-numbers-gamma   
10  commons-numbers          commons-numbers-gamma   
11  commons-numbers          commons-numbers-gamma   
12  commons-numbers          commons-numbers-gamma   
13  commons-numbers          commons-numbers-gamma   
14  commons-numbers          commons-numbers-gamma   
15  commons-numbers          commons-numbers-gamma   
16  commons-numbers          c

                                                class  AssertionRoulette  \
0   org.joda.primitives.collection.impl.TestArrayD...               True   
1   org.joda.primitives.collection.impl.TestArrayB...               True   
2   org.joda.primitives.collection.impl.TestArrayC...               True   
3   org.joda.primitives.collection.impl.TestArrayB...               True   
4   org.joda.primitives.collection.impl.TestArrayF...               True   
5   org.joda.primitives.collection.impl.TestArrayI...               True   
6   org.joda.primitives.collection.impl.TestArrayS...               True   
7   org.joda.primitives.collection.impl.TestArrayL...               True   
8   org.joda.primitives.list.impl.TestImmutableArr...               True   
9     org.joda.primitives.list.impl.TestArrayLongList               True   
10   org.joda.primitives.list.impl.TestArrayShortList               True   
11    org.joda.primitives.list.impl.TestArrayByteList               True   
12  org.joda

       project              module                                    commit  \
0    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
1    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
2    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
3    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
4    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
5    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
6    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
7    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
8    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
9    spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
10   spring-ws  spring-ws-security  a7e5aeb68c9e5dffe7a21069ad2d1d0330c00c60   
11   spring-ws  spring-ws-security  a7e5

                 project  module                                    commit  \
0   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
1   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
2   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
3   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
4   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
5   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
6   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
7   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
8   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
9   spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
10  spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6b72de55fb0   
11  spring-data-keyvalue     NaN  eee744f2f631881f3b50e694af68a6

~~~~~~~~~~~~~~~~~~ jackson-databind ~~~~~~~~~~~~~~~~
             project  module                                    commit  \
0   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
1   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
2   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
3   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
4   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
5   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
6   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
7   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
8   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
9   jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
10  jackson-databind     NaN  49e1c0e457beeead0525eda4e8214be5bb880287   
11  jackson-databind     NaN  49e1c0e457beeead0525eda4e8214

...

                                                class  AssertionRoulette  \
0                           org.joda.time.TestMinutes               True   
1                            org.joda.time.TestMonths               True   
2                org.joda.time.TestDateTimeComparator               True   
3                   org.joda.time.TestAbstractPartial               True   
4                     org.joda.time.TestDurationField               True   
5             org.joda.time.TestBaseSingleFieldPeriod               True   
6                        org.joda.time.TestPeriodType               True   
7                             org.joda.time.TestYears               True   
8                     org.joda.time.TestDateTimeUtils               True   
9                              org.joda.time.TestDays               True   
10                org.joda.time.TestDateTimeFieldType               True   
11                     org.joda.time.TestDateTimeZone               True   
12          

                                              class   LOC     HALSTEAD  RFC  \
0                org.apache.commons.cli.BasicParser    51   395.263088    1   
1       org.apache.commons.cli.PatternOptionBuilder   207  1427.560520   14   
2                    org.apache.commons.cli.Options   367  2152.107076   62   
3                     org.apache.commons.cli.Option  1015  5848.930497   87   
4                org.apache.commons.cli.CommandLine   425  2501.483755   67   
5                org.apache.commons.cli.OptionGroup   179   907.577133   23   
6                  org.apache.commons.cli.GnuParser   115   936.335990   10   
7              org.apache.commons.cli.DefaultParser   707  3942.976244  145   
8              org.apache.commons.cli.OptionBuilder   396  2120.847883   34   
9                       org.apache.commons.cli.Util    72   469.245597   10   
10               org.apache.commons.cli.PosixParser   294  2476.317139   47   
11             org.apache.commons.cli.HelpFormatter 

                                                class   LOC     HALSTEAD  RFC  \
0                   io.undertow.util.SimpleObjectPool    93   603.105636    5   
1   io.undertow.util.SameSiteNoneIncompatibleClien...   157  1475.611203   50   
2                        io.undertow.util.LocaleUtils    79   528.944155   14   
3                        io.undertow.util.PathMatcher   249  1742.993955   58   
4                  io.undertow.util.HeaderTokenParser   143   836.613478   14   
5                         io.undertow.util.HttpString   382  2600.888455   71   
6                          io.undertow.util.HeaderMap   874  3217.381918  155   
7                           io.undertow.util.URLUtils   352  2656.121043   35   
8                       io.undertow.util.PathTemplate   379  2514.186775   39   
9                       io.undertow.util.HeaderValues   582  2657.634435  115   
10                        io.undertow.util.FlexBase64  1871  9842.726017   43   
11                         i

   project    module                                    commit  \
0    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
1    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
2    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
3    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
4    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
5    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
6    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
7    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
8    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
9    zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
10   zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
11   zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
12   zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
13   zxing      core  67138cedc8cd2c8db2158be5da402ba0e2e1b000   
14   zxing

IOError: [Errno 2] File mutation/commons-fileupload.csv does not exist: 'mutation/commons-fileupload.csv'

In [ ]:
df_test_smells